# Imports

In [ ]:
from arcgis.gis import GIS
import copy
import json
from json import dumps
from json import JSONDecoder
from arcgis.mapping import WebMap
from arcgis.apps import workforce
import requests

# Login

In [ ]:
#gis = GIS('http://portalgestor.esri.co/portal', 'sgcoficial', 'Sgcoficial$2020')
# gis = GIS('http://editorcatastral2dev.esri.co/portal', 'ejecutor2', 'Esrico2021$') # El servicio de WFM debe estar compartido para que los ejecutores lo puedan ver
gis = GIS(profile="sgc20")

In [ ]:
class Utils(object):
    trace_messages = ""

    def __init__(self, gis_obj):
        self.gis = gis_obj

    def addMessage(self, message, tipo="MESSAGE"):
        '''Agrega el mensaje enviado a la traza.
        Parametros:
        - message: Cadena de texto con informacion relevante al funcionamiento del servicio
        - {tipo}: tipo de mensaje MESSAGE/WARNING/ERROR (por defecto MESSAGE)
        '''
        if tipo == "MESSAGE":
            print("{}".format(message))
        elif tipo == "WARNING":
            print("{}".format(message))
        elif tipo == "ERROR":
            print("{}".format(message))
        self.trace_messages += "{}\r\n".format(message)

    def ObtenerParametros(self, where_codigo="1=1"):
        """Obtener parametros configurados en la tabla de configuracion del sistema
        Parametros:
        - {where_codigo}: codigos para hacer la consulta solamente por esos codigos (Ej: where_codigo='CODIGO1','CODIGO2','CODIGO3')
        """    
        try:        
            #ACA SE USA EL NOMBRE DE LA CREDENCIAL ALMACENADA EN EL OS
            self.addMessage("Obteniendo parámetros....")
            search_result = self.gis.content.search('configuracion','Feature Layer')
            config_item = search_result[0]            
            tabla_conf = config_item.tables[0]
            query_result = tabla_conf.query(where=where_codigo, out_fields='*')
            listado = {}
            for feature in query_result.features:    
                opcion = feature.attributes["opcion"]
                valor = feature.attributes["valor"]    
                listado[str(opcion)]=valor
            return listado
        except Exception as ex:            
            self.addMessage(message="Error obteniendo los parametros : {}".format(str(ex)), tipo="ERROR")
            return None
    
    def getTableOrLayerWorkSpace(self,nombre_objeto,version_comparar=None):
        try:
            mensaje = ""
            objeto = None
            version = None
            aprx = arcpy.mp.ArcGISProject("CURRENT")
            if aprx is None:
                mensaje = "Error: El proyecto de ArcGIS Pro es nulo"
            else:
                mapa_activo = aprx.activeMap                    
                if mapa_activo is not None:
                    if mapa_activo.name.find("JOB_")!=-1:
                        lista_tablas = mapa_activo.listTables()
                        if len(lista_tablas)>0:
                            for tabla in lista_tablas:
                                if tabla.name.upper() == nombre_objeto.upper():
                                    version = tabla.connectionProperties["connection_info"]["version"]
                                    if version_comparar is not None:
                                        if version==version_comparar:
                                            objeto = tabla
                                            break
                                        else:
                                            mensaje = "La version {} no corresponde con el tramite {}.".format(version,version_comparar)
                                    else:
                                        objeto = tabla
                                        break

                        if objeto is None:
                            lista_layers = mapa_activo.listLayers()
                            if len(lista_layers)>0:
                                for layer in lista_layers:
                                    if layer.name.upper() == nombre_objeto.upper():
                                        version = tabla.connectionProperties["connection_info"]["version"]
                                        if version_comparar is not None:
                                            if version==version_comparar:
                                                objeto = layer
                                                break
                                            else:
                                                mensaje = "La version {} no corresponde con el tramite {}.".format(version,version_comparar)
                                        else:                                            
                                            objeto = layer
                                            break
                    else:
                        mensaje = "Error: El mapa activo no es de un job de WorkFlow"
                else:
                    mensaje = "Error: No hay un mapa activo en el proyecto"
            if objeto is None:
                self.addMessage(message=mensaje, tipo="ERROR")
            return objeto,version,mensaje
        except Exception as ex:            
            self.addMessage(message="Error obteniendo objeto {} del workspace: {}".format(nombre_objeto,str(ex)), tipo="ERROR")
            return objeto,version,mensaje

    def getTramiteMapa(self,tabla_cat_mutaciones):
        try:
            numerotramite = None
            codigopredial = None
            tipotramite = None
            tipomutacion = None
            subtipomutacion = None
            codigos_asociados = None
            version = None
            tabla_encontrada = False       
            cat_mutaciones_object,version,mensaje = self.getTableOrLayerWorkSpace(tabla_cat_mutaciones,None)            
            if cat_mutaciones_object is None:
                self.addMessage(message=mensaje, tipo="ERROR")
            else:                
                tramiteEncontrado = False
                with arcpy.da.SearchCursor(in_table=cat_mutaciones_object, field_names=["numero_tramite","codigo_inicial","tipotram","tipomut","subtipomut","codigos_asoc1"],sql_clause=(None, 'ORDER BY OBJECTID DESC')) as cursor:
                    for row in cursor:
                        tramite = row[0]
                        if version.find(tramite)!=-1:
                            tramiteEncontrado = True
                            codigopredial = row[1]
                            tipotramite = row[2]
                            tipomutacion = row[3]
                            subtipomutacion = row[4]  
                            codigos_asociados = row[5]                                      
                            numerotramite = tramite 
                            self.addMessage("Tramite encontrado:{}, codigo predial:{}".format(tramite,codigopredial))
                            break
                if tramiteEncontrado == False:                                
                    self.addMessage(message="Tramite no encontrado en este mapa", tipo="ERROR")
            return numerotramite,codigopredial,tipotramite,tipomutacion,subtipomutacion,version, codigos_asociados
        except Exception as ex:            
            self.addMessage(message="Error getTramiteMapa. {}".format(str(ex)), tipo="ERROR")
            return None,None,None,None,None,None


In [ ]:
utilsSGC = Utils(gis)
gWFM_SERVICE = "WFM_SERVICE"
REGISTRO_MUTACIONES = "REGISTRO_MUTACIONES"
dictParams = utilsSGC.ObtenerParametros()
wfm_service = dictParams[gWFM_SERVICE]
def getUserGroups(users_groups):
    wfm_service = dictParams[gWFM_SERVICE]
    search_result = gis.content.get(wfm_service)
    if search_result:
        url_wfm_com_operation = search_result.url+"/community/{0}".format(users_groups)
        thetoken = gis._con.token
        parametros = { "f": "json", "token": thetoken }
        r = requests.post(url=url_wfm_com_operation, data=parametros)
        resultjson = json.loads(r.text)
        return resultjson         
    else:
        print("No se encuentra el servicio {}".format(wfm_service))
    return None

In [ ]:
search_result_wfm_service = gis.content.get(wfm_service)
groups_wfm = getUserGroups("groups")
ejecutores = []
for group in groups_wfm["groups"]:
    if group["name"] == "Ejecutores":
        for user in group["users"]:
            ejecutores.append(user)
        break
ejecutores

# Search Webmap template - Variables

In [ ]:
wmTemplate = "6c75cbb0eeff41e7a9469975e3c6c8e2"
#search_result = gis.content.search("{}".format("TEMPLATE EJECUTORES CAMPO"), "Web Map")
#MAPA ORIGINAL renombrado como "LADM_COL_OC_1_0_22_bk"
#search_result = gis.content.search("{}".format("14fd0e42d87a4f8c95ce5b0382756753"), "Web Map")
search_result = gis.content.search("{}".format(wmTemplate), "Web Map")
webmap_template = search_result[0]
#webmap_template

In [ ]:
numerotramite = "08001-2021-00077"
SGC_CAMPO_EJECUTOR = "TRABAJO_CAMPO"
nomservicio = "{0}".format(numerotramite).replace(".", "_")
current_user = gis.users.me.username
usuarioAsignado = "ejecutor1"
item_copy = None
title_wm = nomservicio
search_result_copy = gis.content.search(query="title:{}".format(title_wm), category_filters="Web Map")
search_result_copy

# GenerarWmPublicar (item.copy)

In [ ]:
def GenerarWmPublicar(self, nomservicio, wmTemplate):
    utilsSGC.addMessage("Inicio GenerarWmPublicar: " + nomservicio)
    item_result = None
    search_result = gis.content.search("{}".format(wmTemplate), "Web Map")
    webmap_template = search_result[0]
    item_copy = webmap_template.copy(title=nomservicio)
    item_move = item_copy.move(folder=SGC_CAMPO_EJECUTOR)
    item_result = gis.content.get(item_move["itemId"])
    #Asignar extent al mapa copiado:
    extent_wm = webmap_template.extent
    update_parameters = {'extent': extent_wm}
    item_result.update(item_properties=update_parameters)
    utilsSGC.addMessage("WebMap creado")

    return item_result

# GenerarWebMap (save)

In [ ]:
def GenerarWebMap(wmName, wmTemplate):
    search_result = gis.content.search("{}".format(wmTemplate), "Web Map")
    webmap_template = search_result[0]        
    item_result = None
    search_result_copy = gis.content.search(query="title:{}".format(wmName), category_filters="Web Map")
    if len(search_result_copy) == 0 or (len(search_result_copy)==1 and search_result_copy[0].title != wmName):
        pass
    elif len(search_result_copy) > 1:
        for item in search_result_copy:
            if item.title == wmName:
                item_result = item
                break
    if item_result:
        utilsSGC.addMessage("Ya existe el webmap "+wmName)
    else:
        utilsSGC.addMessage("El WebMap no existe, creandolo")
        wm = WebMap(webmap_template)
        tk = "Offline,Collector"
        web_map_properties = {'typeKeywords': tk, 'title': wmName, 'snippet': wmName, 'tags': wmName}
        item_result = wm.save(item_properties=web_map_properties, folder=SGC_CAMPO_EJECUTOR)
        utilsSGC.addMessage("WebMap creado")

    return item_result

# PublicarWebMap (save/item.copy)

In [ ]:
def PublicarWebMap(wmTemplate, nombreServicio, usuarioAsignado):
    """Generar el nuevo webmap correspondiente al tramite
    Parametros:
    - wmTemplate: Nombre del WebMap usado como plantilla
    - nombreServicio: Nombre que tendra el WebMap
    - usuarioAsignado: Nombre del usuario ejecutor asignado
    """
    utilsSGC.addMessage("Inicio PublicarWebMap:")
    result = False
    try:                   
        # wmItem = GenerarWmPublicar(nombreServicio, wmTemplate)
        search_result = gis.content.search("{}".format(wmTemplate), "Web Map")
        webmap_template = search_result[0]
        wm = WebMap(webmap_template)
        tk = "Offline,Collector"
        web_map_properties = {'typeKeywords': tk, 'title': nombreServicio, 'snippet': nombreServicio, 'tags': nombreServicio}
        wmItem = wm.save(item_properties=web_map_properties, folder=SGC_CAMPO_EJECUTOR)
        utilsSGC.addMessage("WebMap creado")
        if wmItem is None:
            raise Exception("Error creando copia de webmap {}".format(wmTemplate))            
        #Modificar propiedades del wm copiado:
        pubMapJson = wmItem.get_data(try_json=True)
        lyrFilter = "usuario_portal = '{0}'".format(usuarioAsignado)
        fldMapJsonCopy = copy.deepcopy(pubMapJson)
        title_layer = "LADM_COL_OC_1_0_22 - LC_UnidadEspacial"
        lyrT = [l for l in pubMapJson['operationalLayers'] if l['title']== title_layer]
        if len(lyrT) > 0:
            tZoneLyr = lyrT[0]
            if 'layerDefinition' in tZoneLyr:
                z = [[x, y] for x, y in enumerate(pubMapJson['operationalLayers']) if y['title'] == title_layer][0]
                fldMapJsonCopy['operationalLayers'][z[0]]['layerDefinition']['definitionExpression'] = lyrFilter
        wmItem.update(item_properties={'text':json.dumps(fldMapJsonCopy)})
        result = True            
    except Exception as e:
        utilsSGC.addMessage("Error publicando webmap "+ str(e), "ERROR")
    return result

# Publicar WebMap

In [ ]:
#item_result = GenerarWebMap(title_wm, wmTemplate)
item_result = PublicarWebMap(wmTemplate, title_wm, usuarioAsignado)
item_result

# Modificar propiedades del wm copiado

In [ ]:
pubMapJson_template = webmap_template.get_data(try_json=True)
#https://community.esri.com/t5/arcgis-api-for-python-questions/filter-layer-inside-webmap/td-p/806503
#Obtener el webmap copiado
timeZoneWebMap = item_result
#Get the webmap json
pubMapJson = item_result.get_data(try_json=True)
# Filter Expression to apply to layer
usuarioasignado = "ejecutor1"
lyrFilter = "usuario_portal = '{0}'".format(usuarioasignado)

In [ ]:
#List comprehension to find the one layer you need to modify by title
#Here you could loop on all layers to modify the filter too.
title_layer = "LADM_COL_OC_1_0_22 - LC_UnidadEspacial"
lyrT = [l for l in pubMapJson['operationalLayers'] if l['title']== title_layer]
#lyrT

In [ ]:
#Make a deepcopy of the json to modify
fldMapJsonCopy = copy.deepcopy(pubMapJson)
if len(lyrT) > 0:
    tZoneLyr = lyrT[0]
    if 'layerDefinition' in tZoneLyr:
       z = [[x, y] for x, y in enumerate(pubMapJson['operationalLayers']) if y['title'] == title_layer][0]
       fldMapJsonCopy['operationalLayers'][z[0]]['layerDefinition']['definitionExpression'] = lyrFilter

In [ ]:
#Save the changes to the Webmap
timeZoneWebMap.update(item_properties={'text':dumps(fldMapJsonCopy)})
timeZoneWebMap

# Asignación en Workforce

In [ ]:
def crearAsignacionWorkForce(assignedTo, createdBy, workforce_itemid_project, location, jobid, descripcion, nrotramite):
    resultado = False
    try:
        print("Creando asignacion workforce")
        # poligonmap = self.ObtenerExtentJob(jobid, 102100)
        # puntocentroide = poligonmap.centroid
        # X:-74.82530787377105, Y:10.995190070952786
        puntocentroide = {"X":-74.82530787377105, "Y":10.995190070952786}
        workforce_project_item = gis.content.get(workforce_itemid_project)
        project = workforce.Project(item=workforce_project_item)
        worker = project.workers.get(user_id=assignedTo.lower())
        dispatcher = project.dispatchers.get(user_id=createdBy.lower())
        tipo = project.assignment_types.get(name="Ejecutores")
        print("Ejecutando la asignacion en workforce...")
        project.assignments.add(
            assignment_type=tipo,
            status="assigned",
            assigned_date=datetime.datetime.now(),
            worker=worker,
            dispatcher=dispatcher,
            location=location,
            description=descripcion,
            # geometry={'x': puntocentroide.X, 'y': puntocentroide.Y},
            geometry={'x': puntocentroide["X"], 'y': puntocentroide["Y"]},
            work_order_id=nrotramite)
        print("Asignacion realizada satisfactoriamente")
        resultado = True
    except Exception as e:
        print("Ha ocurrido un error en la asignacion a Workforce: " + str(e), "ERROR")
    return resultado

In [ ]:
jobid = -1
codigopredial = "080010103000004060036000000000"
nroManzana = ""
nroPredio = ""
tipopredio = ""
esPh = codigopredial[21:22]
if esPh in ("9", "5", "6"):
    nroPredio = "Nro de Predio:" + codigopredial[17:30]
else:
    nroPredio = "Nro de Predio:" + codigopredial[17:21]

tp = codigopredial[5:7]
if tp == "00":
    tipopredio = "RURAL"
    nroManzana = "Nro de Sector: {}, Nro de Vereda: {}".format(codigopredial[7:9], codigopredial[13:17])
else:
    tipopredio = "URBANO"
    nroManzana = "Nro de Sector: {}, Nro de Manzana: {}".format(codigopredial[7:9], codigopredial[13:17])
idWebMap = item_result.id
workforce_itemid_project = "211e695a5a0048bf915cf8963ce4a38b"
location = "Nro de tramite:" + numerotramite + ", " + nroManzana + ", " + nroPredio
descripcion = "webmapid:" + idWebMap + "|predio:" + codigopredial + "|ambito:" + tipopredio + "|proceso:CON|nrotramite:" + numerotramite
AsignacionWorkForce = crearAsignacionWorkForce(usuarioAsignado, current_user, workforce_itemid_project, location, jobid, descripcion, numerotramite)
if AsignacionWorkForce:
    result = True
    print("asignacion workforce satisfactoria")
else:
    result = False
    print("Fallas en asignacion workforce")